In [1]:
'''
Gene/protein tagging from lists of gene and protein names

Requires ProteinDict.csv, GeneSynonymDict.csv, GeneDictList3, Lowercase_Gene_Dict.csv
'''

import pandas as pd
import enchant
import re

#import series csv as dataframe
df = pd.read_csv (r'Rand400Series-Final.csv', encoding= 'unicode_escape')

#import protein list
ProteinDictdf = pd.read_csv (r'ProteinDict.csv')
ProteinDictList = list(ProteinDictdf.Protein)


#Make list of proteins that contain spaces
#requires different tagging method to entities without spaces
SpacedProteinList = []
for i in ProteinDictList:
    if ' ' in i:
        SpacedProteinList.append(i)
        
#Make list of proteins that don't contain spaces
UnspacedProteinList = []
for i in ProteinDictList:
    if ' ' not in i:
        UnspacedProteinList.append(i)

#import synonym list
SynonymDictdf = pd.read_csv (r'GeneSynonymDict.csv')
SynonymDictList = list(SynonymDictdf.Gene)

#import gene list
GeneDictdf = pd.read_csv (r'GeneDictList3.csv')
GeneDictList = list(GeneDictdf.Gene)

#import lowercase gene list
LowerGeneDictdf = pd.read_csv (r'Lowercase_Gene_Dict.csv')
LowerGeneDictList = list(LowerGeneDictdf.Gene)


#remove english words from lowercase gene list
NewLowerGeneDictList = []

LowerGeneDictList = [str(r) for r in LowerGeneDictList]

d = enchant.Dict("en_US")
for i in LowerGeneDictList:
    if d.check(i) == False:
        NewLowerGeneDictList.append(i)

        
#define gene tagging function
format_gene = "<GENE>{}<GENE>".format #this is how you

def TagGene(input):
    TaggedText = " ".join(format_gene(word) if word in GeneDictList else word for word in input.split())
    return TaggedText


#define synonym tagging function
format_synonym = "<SYNONYM>{}<SYNONYM>".format #this is how you

def TagSynonym(input):
    TaggedText = " ".join(format_synonym(word) if word in SynonymDictList else word for word in input.split())
    return TaggedText


#define protein tagging functions (spaced and unspaced)
format_protein = "<PROTEIN>{}<PROTEIN>".format #this is how you

def TagSpacedProtein(input):
    for i in SpacedProteinList:
        if i in input:
            input = input.replace(i, '<PROTEIN>'+i+'<PROTEIN>')        
    return input

def TagUnspacedProtein(input):
    TaggedText = " ".join(format_protein(word) if word in UnspacedProteinList else word for word in input.split())
    return TaggedText


#define lowercase gene tagging function
format_lowergene = "<LGENE>{}<LGENE>".format #this is how you

def TagLowerGene(input):
    TaggedText = " ".join(format_lowergene(word) if word.lower() in NewLowerGeneDictList else word for word in input.split())
    return TaggedText


#format summary and title column
df['summary'] = df['summary'].astype(str)
df['summary'] = df['summary'].replace('(', '')
df['summary'] = df['summary'].replace(')', '') 
df['title'] = df['title'].astype(str)


#Tag title
df["TaggedTitle"] = df['title'].apply(lambda row: TagGene(row))
df["TaggedTitle"] = df['TaggedTitle'].apply(lambda row: TagSynonym(row))
df["TaggedTitle"] = df['TaggedTitle'].apply(lambda row: TagUnspacedProtein(row))
df["TaggedTitle"] = df['TaggedTitle'].apply(lambda row: TagSpacedProtein(row))
df["TaggedTitle"] = df['TaggedTitle'].apply(lambda row: TagLowerGene(row))



#Tag text with gene names
df["TaggedSummary"] = df['summary'].apply(lambda row: TagGene(row))

#Tag text with synonyms names
df["TaggedSummary"] = df['TaggedSummary'].apply(lambda row: TagSynonym(row))

#Tag text with single word protein names
df["TaggedSummary"] = df['TaggedSummary'].apply(lambda row: TagUnspacedProtein(row))

#Tag text with spaced protein names
df["TaggedSummary"] = df['TaggedSummary'].apply(lambda row: TagSpacedProtein(row))

#Tag text with lowercase gene names
df["TaggedSummary"] = df['TaggedSummary'].apply(lambda row: TagLowerGene(row))


#convert tagged summary to string
df['TaggedSummary'] = df['TaggedSummary'].astype(str)



#extract tagged genes
GenePattern ='<GENE>(.*?)<GENE>'
def GeneExtract(input):
    output = re.findall(GenePattern,input)
    return output

#Tag text with gene names
df["TaggedGenes"] = df['TaggedSummary'].apply(lambda row: GeneExtract(row))

#extract case insensitive tagged genes
LGenePattern ='<LGENE>(.*?)<LGENE>'
def LGeneExtract(input):
    output = re.findall(LGenePattern,input)
    return output

df["TaggedLowerGenes"] = df['TaggedSummary'].apply(lambda row: LGeneExtract(row))

#extract tagged synonyms
SynonymPattern ='<SYNONYM>(.*?)<SYNONYM>'
def SynonymExtract(input):
    output = re.findall(SynonymPattern,input)
    return output

df["TaggedSynonym"] = df['TaggedSummary'].apply(lambda row: SynonymExtract(row))

#extract tagged proteins
ProteinPattern ='<PROTEIN>(.*?)<PROTEIN>'
def ProteinExtract(input):
    output = re.findall(ProteinPattern,input)
    return output

df["TaggedProtein"] = df['TaggedSummary'].apply(lambda row: ProteinExtract(row))


#remove duplicates
df['TaggedGenes'] = df['TaggedGenes'].apply(lambda row: list(set(row)))

df['TaggedLowerGenes'] = df['TaggedLowerGenes'].apply(lambda row: list(set(row)))

df['TaggedSynonym'] = df['TaggedSynonym'].apply(lambda row: list(set(row)))

df['TaggedProtein'] = df['TaggedProtein'].apply(lambda row: list(set(row)))


#export tagged summary to csv
df.to_csv(r'TaggedSeries1.csv', index = False)

In [2]:
df.head()

,id,uid,accession,title,summary,question,answer,BioBERT,TaggedTitle,TaggedSummary,TaggedGenes,TaggedLowerGenes,TaggedSynonym,TaggedProtein
0,3641,200141034,GSE141034,Rph1 coordinates transcription of ribosomal pr...,Coordinated regulation of ribosomal RNA (rRNA)...,name of all genes,NaN,"Answer: ""deletion of rph1""",Rph1 coordinates transcription of ribosomal pr...,Coordinated regulation of ribosomal RNA (rRNA)...,[TORC1],[],[],[]
1,2302,200148962,GSE148962,N6-methyladenosine (m6A) profiling and transla...,To investigate the role of METTL3-mediated m6A...,name of all genes,NaN,"Answer: ""mettl3""",N6-methyladenosine (m6A) profiling and transla...,To investigate the role of METTL3-mediated m6A...,[METTL3],[],[],[]
2,9848,200083769,GSE83769,Dnmt3a Regulates T-cell Development and Suppre...,This SuperSeries is composed of the SubSeries ...,name of all genes,NaN,"Answer: ""[CLS] which gene is knocked ?""",<PROTEIN>Dnmt3a<PROTEIN> Regulates T-cell Deve...,This SuperSeries is composed of the SubSeries ...,[],[],[],[]
3,9824,200090024,GSE90024,Transcriptome analysis to investigate the onco...,Our study shows that TGF-ÃÂ² signaling promot...,name of all genes,NaN,"Answer: ""snail""",Transcriptome analysis to investigate the onco...,Our study shows that TGF-ÃÂ² signaling promot...,[],[],[],[Smad7]
4,6460,200121660,GSE121660,Arnt2 tunes activity-dependent gene expression...,"ChIP-, ATAC-, and RNA-seq data Neuronal activi...",name of all genes,NaN,"Answer: ""[CLS]""",<LGENE>Arnt2<LGENE> tunes activity-dependent g...,"ChIP-, ATAC-, and RNA-seq data Neuronal activi...","[NPAS4, ARNT2]",[NCoR2],[],[]


In [3]:
import pandas as pd

In [4]:
df.head(3)

,id,uid,accession,title,summary,question,answer,BioBERT,TaggedTitle,TaggedSummary,TaggedGenes,TaggedLowerGenes,TaggedSynonym,TaggedProtein
0,3641,200141034,GSE141034,Rph1 coordinates transcription of ribosomal pr...,Coordinated regulation of ribosomal RNA (rRNA)...,name of all genes,NaN,"Answer: ""deletion of rph1""",Rph1 coordinates transcription of ribosomal pr...,Coordinated regulation of ribosomal RNA (rRNA)...,[TORC1],[],[],[]
1,2302,200148962,GSE148962,N6-methyladenosine (m6A) profiling and transla...,To investigate the role of METTL3-mediated m6A...,name of all genes,NaN,"Answer: ""mettl3""",N6-methyladenosine (m6A) profiling and transla...,To investigate the role of METTL3-mediated m6A...,[METTL3],[],[],[]
2,9848,200083769,GSE83769,Dnmt3a Regulates T-cell Development and Suppre...,This SuperSeries is composed of the SubSeries ...,name of all genes,NaN,"Answer: ""[CLS] which gene is knocked ?""",<PROTEIN>Dnmt3a<PROTEIN> Regulates T-cell Deve...,This SuperSeries is composed of the SubSeries ...,[],[],[],[]


In [5]:

df['ExtractedGenes'] = df['TaggedLowerGenes'] + df['TaggedGenes']
df.head(10)

,id,uid,accession,title,summary,question,answer,BioBERT,TaggedTitle,TaggedSummary,TaggedGenes,TaggedLowerGenes,TaggedSynonym,TaggedProtein,ExtractedGenes
0,3641,200141034,GSE141034,Rph1 coordinates transcription of ribosomal pr...,Coordinated regulation of ribosomal RNA (rRNA)...,name of all genes,NaN,"Answer: ""deletion of rph1""",Rph1 coordinates transcription of ribosomal pr...,Coordinated regulation of ribosomal RNA (rRNA)...,[TORC1],[],[],[],[TORC1]
1,2302,200148962,GSE148962,N6-methyladenosine (m6A) profiling and transla...,To investigate the role of METTL3-mediated m6A...,name of all genes,NaN,"Answer: ""mettl3""",N6-methyladenosine (m6A) profiling and transla...,To investigate the role of METTL3-mediated m6A...,[METTL3],[],[],[],[METTL3]
2,9848,200083769,GSE83769,Dnmt3a Regulates T-cell Development and Suppre...,This SuperSeries is composed of the SubSeries ...,name of all genes,NaN,"Answer: ""[CLS] which gene is knocked ?""",<PROTEIN>Dnmt3a<PROTEIN> Regulates T-cell Deve...,This SuperSeries is composed of the SubSeries ...,[],[],[],[],[]
3,9824,200090024,GSE90024,Transcriptome analysis to investigate the onco...,Our study shows that TGF-ÃÂ² signaling promot...,name of all genes,NaN,"Answer: ""snail""",Transcriptome analysis to investigate the onco...,Our study shows that TGF-ÃÂ² signaling promot...,[],[],[],[Smad7],[]
4,6460,200121660,GSE121660,Arnt2 tunes activity-dependent gene expression...,"ChIP-, ATAC-, and RNA-seq data Neuronal activi...",name of all genes,NaN,"Answer: ""[CLS]""",<LGENE>Arnt2<LGENE> tunes activity-dependent g...,"ChIP-, ATAC-, and RNA-seq data Neuronal activi...","[NPAS4, ARNT2]",[NCoR2],[],[],"[NCoR2, NPAS4, ARNT2]"
5,2335,200162272,GSE162272,Co-dependency for MET and FGFR1 in basal tripl...,Triple-negative breast cancer (TNBC) is a hete...,name of all genes,NaN,"Answer: ""met""",Co-dependency for <GENE>MET<GENE> and <GENE>FG...,Triple-negative breast cancer (TNBC) is a hete...,[FGFR1],[],[],[],[FGFR1]
6,7671,200120826,GSE120826,"Transcriptional profiles of Sox4 cKO, Sox11 cK...",Transcriptional profiling of mouse E16 epiderm...,name of all genes,NaN,"Answer: ""[CLS]""",Transcriptional profiles of <LGENE>Sox4<LGENE>...,Transcriptional profiling of mouse <PROTEIN>E1...,[],[],[],[E16],[]
7,14208,200064101,GSE64101,Pit-1 overexpression in the human breast adeno...,The aim of the study is to evaluate Pit-1-indu...,name of all genes,NaN,"Answer: ""pit""",Pit-1 overexpression in the human breast adeno...,The aim of the study is to evaluate Pit-1-indu...,[],[],[],[pituitary],[]
8,12279,200081258,GSE81258,Nfib promotes Metastasis through a Widespread ...,This SuperSeries is composed of the SubSeries ...,name of all genes,NaN,"Answer: ""[CLS] which gene is knocked ?""",<LGENE>Nfib<LGENE> promotes Metastasis through...,This SuperSeries is composed of the SubSeries ...,[],[],[],[],[]
9,17626,200021900,GSE21900,Expression profiling of the Otx2 CKO retina,"In the vertebrate retina, the Otx2 transcripti...",name of all genes,NaN,"Answer: ""otx2""",Expression profiling of the <LGENE>Otx2<LGENE>...,"In the vertebrate retina, the <LGENE>Otx2<LGEN...",[],[Otx2],[],[],[Otx2]


In [22]:
df.sort_values('id').to_csv('Rand400Series_BioBERT_sorted_id')
df.head(3)

,id,uid,accession,title,summary,question,answer,BioBERT,TaggedTitle,TaggedSummary,TaggedGenes,TaggedLowerGenes,TaggedSynonym,TaggedProtein,ExtractedGenes
0,3641,200141034,GSE141034,Rph1 coordinates transcription of ribosomal pr...,Coordinated regulation of ribosomal RNA (rRNA)...,name of all genes,NaN,"Answer: ""deletion of rph1""",Rph1 coordinates transcription of ribosomal pr...,Coordinated regulation of ribosomal RNA (rRNA)...,[TORC1],[],[],[],[TORC1]
1,2302,200148962,GSE148962,N6-methyladenosine (m6A) profiling and transla...,To investigate the role of METTL3-mediated m6A...,name of all genes,NaN,"Answer: ""mettl3""",N6-methyladenosine (m6A) profiling and transla...,To investigate the role of METTL3-mediated m6A...,[METTL3],[],[],[],[METTL3]
2,9848,200083769,GSE83769,Dnmt3a Regulates T-cell Development and Suppre...,This SuperSeries is composed of the SubSeries ...,name of all genes,NaN,"Answer: ""[CLS] which gene is knocked ?""",<PROTEIN>Dnmt3a<PROTEIN> Regulates T-cell Deve...,This SuperSeries is composed of the SubSeries ...,[],[],[],[],[]


In [20]:
df2 = pd.read_csv('Rand400SeriesBERT_2.csv', encoding= 'unicode_escape')
df2.sort_values('id').head().to_csv('Rand400SeriesBERT_sorted_id')
df2.head(3)

,id,uid,accession,title,summary,All Genes,Manipulated Genes,BERT
0,1063.0,200165074.0,GSE165074,RNA-Seq analyses of isoalted primary cardiac f...,Purpose: The biological and biochemical functi...,"FAM114A1, P60",Fam114a1,fam114a1 - / -
1,16972.0,200036665.0,GSE36665,Gene expression profiling of Wwox targeted in...,The WWOX gene has been implicated in human can...,WWOX,WWOX,wwox
2,16289.0,200042447.0,GSE42447,Activation of the chromatin-remodeling factor ...,We investigate the role of Brg1 and Olig2 duri...,"Brg1, Olig2, RPolII, Smarca4, Olig2",NaN,ol


In [1]:
#df['totalwords'] = df['Manipulated Genes'].str.count(' ') + 1


In [2]:
#df['totalwords']

# Remove stoping words

In [13]:
import nltk
from nltk.corpus import stopwords

In [14]:
df['Manipulated Genes'].apply(lambda row: str(row))

0      Fam114a1 knockout
1          WWOX knockout
2                    nan
3                    nan
4                    nan
             ...        
398      Eri1 deficiency
399                  nan
400        CD4 depletion
401                  nan
402                  nan
Name: Manipulated Genes, Length: 403, dtype: object

In [15]:

stop = stopwords.words('english')

In [16]:
df['Manipulated Genes'] = df['Manipulated Genes'].astype(str)


In [17]:
#Split string in summary column into list of words
def split_string(row):
  return row['Manipulated Genes'].replace('.','').replace(',','').replace('(','').replace(')','').replace("'",' ').split(' ')

df["Manipulated_Genes_list"] = df.apply(split_string, axis=1)

In [18]:
df["Manipulated_Genes_list"]

0      [Fam114a1, knockout]
1          [WWOX, knockout]
2                     [nan]
3                     [nan]
4                     [nan]
               ...         
398      [Eri1, deficiency]
399                   [nan]
400        [CD4, depletion]
401                   [nan]
402                   [nan]
Name: Manipulated_Genes_list, Length: 403, dtype: object

In [19]:
df5 = pd.read_csv('Rand400SeriesBERT_2.csv', encoding= 'unicode_escape')

In [20]:
df5.head(3)

,id,uid,accession,title,summary,All Genes,Manipulated Genes,BERT
0,1063.0,200165074.0,GSE165074,RNA-Seq analyses of isoalted primary cardiac f...,Purpose: The biological and biochemical functi...,"FAM114A1, P60",Fam114a1,fam114a1 - / -
1,16972.0,200036665.0,GSE36665,Gene expression profiling of Wwox targeted in...,The WWOX gene has been implicated in human can...,WWOX,WWOX,wwox
2,16289.0,200042447.0,GSE42447,Activation of the chromatin-remodeling factor ...,We investigate the role of Brg1 and Olig2 duri...,"Brg1, Olig2, RPolII, Smarca4, Olig2",NaN,ol


In [21]:
df5 = df5.fillna('')

In [22]:
df5['Manipilated_BERT'] = df5['Manipulated Genes'].astype(str) + df5['BERT'].astype(str)

In [23]:
df5['Manipilated_BERT'] = df5['Manipilated_BERT'].str.lower()
df5.head(20)

,id,uid,accession,title,summary,All Genes,Manipulated Genes,BERT,Manipilated_BERT
0,1063.0,200165074.0,GSE165074,RNA-Seq analyses of isoalted primary cardiac f...,Purpose: The biological and biochemical functi...,"FAM114A1, P60",Fam114a1,fam114a1 - / -,fam114a1 fam114a1 - / -
1,16972.0,200036665.0,GSE36665,Gene expression profiling of Wwox targeted in...,The WWOX gene has been implicated in human can...,WWOX,WWOX,wwox,wwox wwox
2,16289.0,200042447.0,GSE42447,Activation of the chromatin-remodeling factor ...,We investigate the role of Brg1 and Olig2 duri...,"Brg1, Olig2, RPolII, Smarca4, Olig2",,ol,ol
3,18364.0,200016344.0,GSE16344,Expression in Aneuploid Drosophila S2 Cells,Chromosomal and segmental aneuploidies are usu...,No Genes,,histone 4 at lysine 16,histone 4 at lysine 16
4,967.0,200171784.0,GSE171784,Influence of the aminoglycoside antibiotic apr...,Bacterial populations face the constant threat...,No Genes,,apr,apr
5,13362.0,200069209.0,GSE69209,Gene expression changes induced by expression ...,Thrombopoietin (Thpo) signals via its receptor...,"Thpo, Mpl, TIE2, ESAM1, EPCR",,,
6,3935.0,200143681.0,GSE143681,Gene expression in response to knockdown of GLI1.,GLI1 is a transcription factor correlated to d...,"GLI1, SMARCA2, HHIP","GLI1 , SMARCA2",gli1 and smarca2,"gli1 , smarca2 gli1 and smarca2"
7,12418.0,200079924.0,GSE79924,Gene expression profiling of selenophosphate s...,Selenium (Se) is an important trace element fo...,SPS2,SPS2,sec biosynthesis genes and selenoprotein genes,sps2 sec biosynthesis genes and selenoprotein ...
8,2769.0,200145159.0,GSE145159,Re-assessment of the involvement of Snord115 i...,SNORD115 has been proposed to promote the acti...,"SNORD115, HTR2C",Snord115,snord115,snord115 snord115
9,13749.0,200067915.0,GSE67915,Transcriptome Signature and Regulation in Huma...,Reprogramming of somatic cells produces induce...,"OCT4, SOX2, KLF4, MYC, OSKM, CCNE1",,ccne1,ccne1


In [24]:
#Split string in summary column into list of words
def split_string(row):
  return row['Manipilated_BERT'].replace('.','').replace(',','').replace('(','').replace(')','').replace("'",' ').split(' ')

df5['Manipilated_BERT'] = df5.apply(split_string, axis=1)

In [25]:
df5['Manipilated_BERT'] = df5['Manipilated_BERT'].apply(lambda row: list(set(row)))

In [26]:
df5['Manipilated_BERT'].astype(str)

0                      ['fam114a1', '/', '-']
1                                    ['wwox']
2                                      ['ol']
3      ['at', 'histone', '4', '16', 'lysine']
4                                     ['apr']
                        ...                  
398                              ['', 'eri1']
399                                  ['prc2']
400               ['ifn', 'cd4', '-', 'aza³']
401                                      ['']
402                                      ['']
Name: Manipilated_BERT, Length: 403, dtype: object

In [27]:
df5['new_column'] = df5['Manipilated_BERT'].apply(lambda x: len(str(x).split(' '))) 

In [28]:
df5['new_column'].sum()

975

In [29]:
df5['new_column_manipulated'] = df5['Manipulated Genes'].apply(lambda x: len(str(x).split(' '))) 

In [30]:
df5['new_column_manipulated'].sum()

861

In [31]:
df5['new_column_BERT'] = df5['BERT'].apply(lambda x: len(str(x).split(' '))) 

In [32]:
df5['new_column_BERT'].sum()

752

In [33]:
ratio_manipulated = 861/975 * 100
ratio_manipulated

88.3076923076923

In [34]:
ratio_BERT = 752/975 * 100
ratio_BERT

77.12820512820512

When we compare combine genes from BERT and Manipulated we find that manually extracted genes are 88.31 percent accurate while BERT is only 77.13 percent accurate. Meanwhile BERT is less accurate collective results are significanetelly better